In [ ]:
%sql
SELECT
    aufk.aufnr AS ordem,
    aufk.auart AS tipo_ordem,
    aufk.ktext AS texto_breve,
    aufk.erdat AS data_criacao,
    aufk.bukrs AS empresa,
    aufk.werks AS centro_trabalho,
    afih.equnr AS equipamento,
    iloa.tplnr AS local_instalacao
FROM poseidon_uc.saps4hanarawview.aufk AS aufk
LEFT JOIN poseidon_uc.saps4hanarawview.afih AS afih
    ON afih.aufnr = aufk.aufnr
LEFT JOIN poseidon_uc.saps4hanarawview.iloa AS iloa
    ON iloa.iloan = afih.iloan
WHERE COALESCE(aufk.loekz, '') = ''
  AND aufk.auart IN ('GIA','GAA','GRA')
  AND SUBSTRING(iloa.tplnr, 1, 4) IN ('GHIN','GHEN','GHSM')
ORDER BY aufk.erdat DESC, aufk.aufnr
LIMIT 1000


In [ ]:
%sql
WITH orders_ok AS (
    SELECT DISTINCT afih.aufnr
    FROM poseidon_uc.saps4hanarawview.afih afih
    JOIN poseidon_uc.saps4hanarawview.iloa iloa
      ON iloa.iloan = afih.iloan
    WHERE SUBSTRING(iloa.tplnr, 1, 4) IN ('GHIN','GHEN','GHSM')
),
afih_pick AS (
    SELECT
        afih.aufnr,
        MAX(afih.equnr) AS equnr,
        MAX(afih.iloan) AS iloan
    FROM poseidon_uc.saps4hanarawview.afih afih
    WHERE afih.aufnr IN (SELECT aufnr FROM orders_ok)
    GROUP BY afih.aufnr
)
SELECT
    k.aufnr AS ordem,
    k.auart AS tipo_ordem,
    k.ktext AS texto_breve,
    k.erdat AS data_criacao,
    k.bukrs AS empresa,
    k.werks AS centro_trabalho,
    p.equnr AS equipamento,
    i.tplnr AS local_instalacao
FROM poseidon_uc.saps4hanarawview.aufk k
JOIN orders_ok ok ON ok.aufnr = k.aufnr
LEFT JOIN afih_pick p ON p.aufnr = k.aufnr
LEFT JOIN poseidon_uc.saps4hanarawview.iloa i ON i.iloan = p.iloan
WHERE COALESCE(k.loekz, '') = ''
  AND k.auart IN ('GIA','GAA','GRA')
ORDER BY k.erdat DESC, k.aufnr
LIMIT 1000

